


col# BEFORE YOU START, CHANGE THE RUNTIME TO REQUEST A T4 GPU

In [1]:
# Installs
# You will need at a minimum the following packages. Feel free to install
# additional ones as needed
!pip install google-generativeai
!pip install datasets
!pip install -U bitsandbytes
!pip install transformers
!pip install -U peft
!pip install -U "huggingface_hub[cli]"
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.0/516.0 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.1.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 33.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.1.5
    Uninstalling huggingface_hub-1.1.5:
      Successfully uninstalled huggingface_hub-1.1.5


In [2]:
import google.generativeai as genai
from datasets import Dataset, DatasetDict
import pandas as pd
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, \
    BitsAndBytesConfig, TrainingArguments, pipeline, logging
import torch
from trl import SFTTrainer

In [3]:
# For this code to work, both train_dataset and eval_dataset must both be on the same google drive since this code is running on T4 GPU.
# Loop through text file datasets (training) and cleaning whitespace to "r" read the txt file and produce plain text with no newline characters.
with open("train_dataset.txt", "r") as f:
    qa_pairs_train = [line.strip() for line in f]
# Loop through text file datasets (Validation) and cleaning whitespace to "r" read the txt file and produce plain text with no newline characters.
with open("eval_dataset.txt", "r", encoding="utf-8") as f:
    qa_pairs_val = [line.strip() for line in f]


# Create HuggingFace datasets
dataset_train = Dataset.from_pandas(pd.DataFrame(qa_pairs_train, columns=["text"]))
dataset_val = Dataset.from_pandas(pd.DataFrame(qa_pairs_val, columns=["text"]))
dataset = DatasetDict({"train": dataset_train, "test": dataset_val})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text'],
        num_rows: 210
    })
})


In [4]:
!huggingface-cli login --token

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: write).
The token `mistral` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mistral`


In [5]:
# Load the model -- Skeleton
base_model = "mistralai/Mistral-7B-Instruct-v0.2" # Chosen based on the synthetic Q/A Choice for the assignment.
bnb_config = BitsAndBytesConfig( # Using this website to know how to call quantization configuration: https://huggingface.co/docs/transformers/en/quantization/bitsandbytes
    load_in_4bit=True,

    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained( # Using this website to know how to call quantization configuration: https://huggingface.co/docs/transformers/en/quantization/bitsandbytes
    base_model,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [6]:
# Tokenize the data
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

('<s>', '</s>')

In [25]:
# LoRA config -- Skeleton
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=32, #matrix rank
    lora_alpha=64, # Scaling Factor
    lora_dropout=0.05,  # standard value to avoid overfitting
    bias="none", # avoids updating weights; therefore saving a lot of computing power and memory
    task_type="CAUSAL_LM",  # Causal language modeling, predicts next token in a sequence
    target_modules=[   # Mistral's model layers

        "q_proj", # attention
        "k_proj", # attention
        "v_proj", # attention
        "o_proj", # attention

        "gate_proj", # feed-forward network
        "up_proj", # feed-forward network
        "down_proj", # feed-forward network

        "lm_head", # Gives output as probability
    ],

)
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(


In [31]:
#import wandb
#wandb.login(key="")

#wandb.init(
#    project="ese577-laywer-finetune",
#)

# Hyperparameters -- Skeleton
training_arguments = TrainingArguments(
        warmup_steps=2, # Starts learning rate slow for the first 2 steps for model stability
        per_device_train_batch_size=2, # Batch size for small dataset on limited GPU
        gradient_accumulation_steps=1, # batch accumlation before optimizer updates
        gradient_checkpointing=True, # only saving checkpoints saving memory rather than all steps
        max_steps=500,

        optim="paged_adamw_8bit", # Using AdamW as the optimizer as it is standard and the paged version uses less memory
        learning_rate=2.5e-5, # small lr because dataset is small and Mistral is already pretrained, we don't want to override or potentially mess with the existing learning it has.
        bf16=True,  # Uses less bits so less memory



        #report_to="wandb",
        do_eval=True, # evaluates after training
)

In [24]:
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    args=training_arguments,

)
trainer.train()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/210 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/210 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/210 [00:00<?, ? examples/s]

CommError: Error uploading run: returned error 403: {"data":{"upsertBucket":null},"errors":[{"message":"permission denied","path":["upsertBucket"],"extensions":{"code":"PERMISSION_ERROR"}}]}

In [ ]:
# Save the model
trainer.model.save_pretrained("ESE577_chatbot")
model.config.use_cache = True
model.eval()

In [ ]:
# Run the model locally
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True)
def build_prompt(question):
  prompt = f"<s>[INST]@ESE577. {question}. [/INST]"
  return prompt

while True:
  question = input("Enter your ESE577-related question (hit Enter to exit): ").strip()
  if not question:
    break
  prompt = build_prompt(question)
  answer = pipe(prompt)
  print(answer[0]["generated_text"])
  print()
